# 기본 세팅

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 기본 세팅

In [ ]:
import tensorflow as tf
import PIL
import numpy as np
import os
import cv2
from google.colab.patches import cv2_imshow

In [ ]:
# https://jkisaaclee.kro.kr/keras/facenet/deep%20learning/computer%20vision/2019/10/01/how_to_develop_a_face_recognition_system_using_facenet_in_keras_ko/
# https://machinelearningmastery.com/how-to-develop-a-face-recognition-system-using-facenet-in-keras-and-an-svm-classifier/

In [ ]:
model = tf.keras.models.load_model('/content/drive/MyDrive/A.아이/github/models_folder/facenet/facenet_model.h5')

In [ ]:
print(model.input)
print(model.output)

In [ ]:
!pip install mediapipe

In [ ]:
friends_path_first = '/content/drive/MyDrive/A.아이/images/friends_image/fist_image/'
friends_path_first_name = os.listdir(friends_path_first)

friends_path_second = '/content/drive/MyDrive/A.아이/images/friends_image/second_image/'
friends_path_second_name = os.listdir(friends_path_second)

In [ ]:
import mediapipe as mp

mp_face_detection = mp.solutions.face_detection
image_friends_first_list = []
label_first_list = []
margin = 0.15  # Adjust this value to change the cropping margin

for name in friends_path_first_name :
    img = cv2.imread(friends_path_first + name)
    with mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.5) as face_detection:
            face = face_detection.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
            if face.detections:
                for i, detection in enumerate(face.detections):
                    # Get bounding box coordinates
                    box = detection.location_data.relative_bounding_box
                    ih, iw, _ = img.shape
                    x, y, w, h = int(box.xmin * iw), int(box.ymin * ih), int(box.width * iw), int(box.height * ih)

                    # Calculate margins for x, y, w, h
                    mx = int(w * margin)
                    my = int(h * margin)

                    # Crop the face with margins
                    face_image = img[y+my:y+h-my, x+mx:x+w-mx]
                    if face_image.size != 0:
                        # Resize the image to 105x105
                        face_image = cv2.resize(face_image, (160, 160))
                        face_image = cv2.cvtColor(face_image, cv2.COLOR_BGR2GRAY)
                    else :
                        break
            else :
                print('cant find face')
                continue
    image_friends_first_list.append(face_image)
    label = name.split('_')[0]
    label_first_list.append(label)

for i in image_friends_first_list :
    cv2_imshow(i)


In [ ]:
mp_face_detection = mp.solutions.face_detection
image_friends_second_list = []
label_second_list = []
margin = 0.15  # Adjust this value to change the cropping margin

for name in friends_path_second_name :
    img = cv2.imread(friends_path_second + name)
    with mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.5) as face_detection:
            face = face_detection.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
            if face.detections:
                for i, detection in enumerate(face.detections):
                    # Get bounding box coordinates
                    box = detection.location_data.relative_bounding_box
                    ih, iw, _ = img.shape
                    x, y, w, h = int(box.xmin * iw), int(box.ymin * ih), int(box.width * iw), int(box.height * ih)

                    # Calculate margins for x, y, w, h
                    mx = int(w * margin)
                    my = int(h * margin)

                    # Crop the face with margins
                    face_image = img[y+my:y+h-my, x+mx:x+w-mx]
                    if face_image.size != 0:
                        # Resize the image to 105x105
                        face_image = cv2.resize(face_image, (160, 160))
                        face_image = cv2.cvtColor(face_image, cv2.COLOR_RGB2GRAY)
                    else :
                        break
            else :
                print('cant find face')
                continue
    image_friends_second_list.append(face_image)
    label = name.split('_')[0]
    label_second_list.append(label)


for count,i in enumerate(image_friends_second_list) :
    print(count)
    cv2_imshow(i)


In [ ]:
label_first_list

In [ ]:
image_friends_first_list = np.array(image_friends_first_list)
image_friends_second_list = np.array(image_friends_second_list)

image_friends_first_list.shape, image_friends_second_list.shape

In [ ]:
face_embeding_model = tf.keras.models.load_model('/content/drive/MyDrive/A.아이/github/models_folder/facenet/facenet_model.h5')

## 간단 테스트

In [ ]:
import matplotlib.pyplot as plt

def get_face_embedding_ver1(image, model):
    # Preprocess the image
    image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    image = image.astype('float32') / 255  # Normalize to [0,1]

    # Compute embedding
    embedding = model.predict(image)[0]
    return embedding

def test_model_with_friends_ver1(index, image_friends_first_list, image_friends_second_list, model):
    # Get the embedding for the anchor image
    anchor_embedding = get_face_embedding_ver1(image_friends_first_list[index], model)

    # Variables to keep track of most similar images
    most_similar_images = []  # A list to keep track of the indices of the most similar images
    distances = []  # A list to keep track of the distances

    # Iterate over images to compute distances
    for i, image in enumerate(image_friends_second_list):
        # Get the embedding for this image
        image_embedding = get_face_embedding_ver1(image, model)

        # Compute the distance
        distance = np.linalg.norm(anchor_embedding - image_embedding)
        distances.append(distance)

    # Compute min and max distances
    min_distance = min(distances)
    max_distance = max(distances)

    # Normalize distances and compute similarities
    for i, distance in enumerate(distances):
        # Normalize the distance to [0,1] using the min and max distances
        normalized_distance = (distance - 0) / (max_distance - 0)

        # Convert to percentage
        percentage = (1 - normalized_distance) * 100

        # Compute the similarity (as 1 - normalized Euclidean distance)
        similarity = percentage

        # Add new image
        most_similar_images.append((i, similarity))  # Keeping track of the index and similarity

    # Sort the images by similarity
    most_similar_images.sort(key=lambda x: x[1], reverse=True)

    # Show the anchor image
    plt.figure(figsize=(20, 4))
    plt.subplot(1, 6, 1)
    plt.imshow(image_friends_first_list[index])
    plt.title("Anchor image")
    plt.axis('off')

    # Show the most similar images
    for i, (img_index, similarity) in enumerate(most_similar_images[:5]):
        plt.subplot(1, 6, i+2)
        plt.imshow(image_friends_second_list[img_index])
        plt.title(f'Similarity: {similarity:.5f}%')
        plt.axis('off')
    plt.show()

In [ ]:
test_model_with_friends_ver1(4, image_friends_first_list, image_friends_second_list, face_embeding_model)

# 함수

In [ ]:
import tensorflow as tf
import mediapipe as mp
import numpy as np
import os
import cv2

class FaceEmbedder:
    def __init__(self, model_path: str, margin: float = 0.15) :
        self.margin = margin  # Adjust this value to change the cropping margin
        self.model = tf.keras.models.load_model(model_path)
        self.face_detection = mp.solutions.face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.5)

    def _get_distance(self, embedded_image1, embedded_image2) :
        distance = np.linalg.norm(embedded_image1 - embedded_image2)
        return distance

    def _get_most_similar_vactor(self, distance_list) :
        # Compute min and max distances
        min_distance = min(distance_list)
        max_distance = max(distance_list)
        similar_images = []

        # Normalize distances and compute similarities
        for i, distance in enumerate(distance_list):
            # Normalize the distance to [0,1] using the min and max distances
            normalized_distance = (distance - 0) / (max_distance - 0)

            # Convert to percentage
            percentage = (1 - normalized_distance) * 100

            # Compute the similarity (as 1 - normalized Euclidean distance)
            similarity = percentage

            # Add new image
            similar_images.append((i, similarity))  # Keeping track of the index and similarity

        # Sort the images by similarity in descending order
        similar_images.sort(key=lambda x: x[1], reverse=True)

        # Return the top 5 most similar images (indices and similarities)
        return similar_images[:5]


    def _get_face(self, img) :
        img = img
        face = self.face_detection.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        if not face.detections:
            print('Cant find face')
            return None

        for detection in face.detections:
            # Get bounding box coordinates
            box = detection.location_data.relative_bounding_box
            ih, iw, _ = img.shape
            x, y, w, h = int(box.xmin * iw), int(box.ymin * ih), int(box.width * iw), int(box.height * ih)

            # Calculate margins for x, y, w, h
            mx = int(w * self.margin)
            my = int(h * self.margin)

            # Crop the face with margins
            face_image = img[y+my:y+h-my, x+mx:x+w-mx]
            if face_image.size == 0:
                print('Face image size is null')
                continue
            face_image = cv2.cvtColor(face_image, cv2.COLOR_RGB2GRAY)
            return cv2.resize(face_image, (160, 160))

        return None

    def get_embedded_face(self, face_image) :
        img = cv2.imread(face_image)
        if img is None:
            print(f"Failed to read image: {face_image}")
            return None

        face_image = self._get_face(img)
        if face_image is None:
            return None

        # Preprocess the image
        image = cv2.cvtColor(face_image, cv2.COLOR_GRAY2RGB)
        image = np.expand_dims(image, axis=0)  # Add batch dimension
        image = image.astype('float32') / 255  # Normalize to [0,1]

        # Compute embedding
        return self.model.predict(image)[0]

In [ ]:
embeding = FaceEmbedder(model_path = '/content/drive/MyDrive/A.아이/github/models_folder/facenet/facenet_model.h5')

In [ ]:
import matplotlib.pyplot as plt
import cv2

def test_model_with_friends(index, image_friends_first_list, image_friends_second_list, face_embedder):
    # Get the embedding for the anchor image
    anchor_image = friends_path_first+image_friends_first_list[index]
    anchor_embedding = face_embedder.get_embedded_face(anchor_image)

    # Variables to keep track of most similar images
    distances = []  # A list to keep track of the distances

    # Iterate over images to compute distances
    for image in image_friends_second_list:
        # Get the embedding for this image
        image_embedding = face_embedder.get_embedded_face(friends_path_second+image)

        # Compute the distance
        if image_embedding is not None:
            distance = face_embedder._get_distance(anchor_embedding, image_embedding)
            distances.append(distance)
        else:
            distances.append(float('inf'))  # Handle the case where the face is not detected
    # Get the most similar images
    most_similar_images = face_embedder._get_most_similar_vactor(distances)

    # Show the anchor image
    plt.figure(figsize=(20, 4))
    plt.subplot(1, 6, 1)
    plt.imshow(cv2.cvtColor(cv2.imread(anchor_image), cv2.COLOR_BGR2RGB))
    plt.title("Anchor image")
    plt.axis('off')

    # Show the most similar images
    for i, (img_index, similarity) in enumerate(most_similar_images):
        similar_image = cv2.imread(friends_path_second+image_friends_second_list[img_index])
        plt.subplot(1, 6, i + 2)
        plt.imshow(cv2.cvtColor(similar_image, cv2.COLOR_BGR2RGB))
        plt.title(f'Similarity: {similarity:.5f}%')
        plt.axis('off')
    plt.show()


In [ ]:
test_model_with_friends(21, friends_path_first_name, friends_path_second_name, embeding)

# 정확도

## tesnorflow lite model load function

In [ ]:
import tensorflow as tf
import mediapipe as mp
import numpy as np
import os
import cv2

class FaceEmbedder:
    def __init__(self, model_path: str, margin: float = 0.15) :
        self.margin = margin  # Adjust this value to change the cropping margin

        self.interpreter = tf.lite.Interpreter(model_path=model_path)
        self.interpreter.allocate_tensors()

        self.__input_details = self.interpreter.get_input_details()
        self.__output_details = self.interpreter.get_output_details()

        self.input_index = self.__input_details[0]['index']
        self.output_index = self.__output_details[0]['index']

        self.face_detection = mp.solutions.face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.5)

    def _get_distance(self, embedded_image1, embedded_image2) :
        distance = np.linalg.norm(embedded_image1 - embedded_image2)
        return distance

    def _get_most_similar_vactor(self, distance_list) :
        # Compute min and max distances
        min_distance = min(distance_list)
        max_distance = max(distance_list)
        similar_images = []

        # Normalize distances and compute similarities
        for i, distance in enumerate(distance_list):
            # Normalize the distance to [0,1] using the min and max distances
            normalized_distance = (distance - 0) / (max_distance - 0)

            # Convert to percentage
            percentage = (1 - normalized_distance) * 100

            # Compute the similarity (as 1 - normalized Euclidean distance)
            similarity = percentage

            # Add new image
            similar_images.append((i, similarity))  # Keeping track of the index and similarity

        # Sort the images by similarity in descending order
        similar_images.sort(key=lambda x: x[1], reverse=True)

        # Return the top 5 most similar images (indices and similarities)
        return similar_images[:5]


    def _get_face(self, img) :
        img = img
        face = self.face_detection.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        if not face.detections:
            print('Cant find face')
            return None

        for detection in face.detections:
            # Get bounding box coordinates
            box = detection.location_data.relative_bounding_box
            ih, iw, _ = img.shape
            x, y, w, h = int(box.xmin * iw), int(box.ymin * ih), int(box.width * iw), int(box.height * ih)

            # Calculate margins for x, y, w, h
            mx = int(w * self.margin)
            my = int(h * self.margin)

            # Crop the face with margins
            face_image = img[y+my:y+h-my, x+mx:x+w-mx]
            if face_image.size == 0:
                print('Face image size is null')
                continue
            face_image = cv2.cvtColor(face_image, cv2.COLOR_RGB2GRAY)
            return cv2.resize(face_image, (160, 160))

        return None

    def get_embedded_face(self, face_image) :
        img = cv2.imread(face_image)
        if img is None:
            print(f"Failed to read image: {face_image}")
            return None

        face_image = self._get_face(img)
        if face_image is None:
            return None

        # Preprocess the image
        image = cv2.cvtColor(face_image, cv2.COLOR_GRAY2RGB)
        image = np.expand_dims(image, axis=0)  # Add batch dimension
        image = image.astype('float32') / 255  # Normalize to [0,1]

        # Compute embedding
        self.interpreter.set_tensor(self.input_index, image)
        self.interpreter.invoke()
        embeddings = self.interpreter.get_tensor(self.output_index)

        return embeddings

## 정확도 테스트 function

In [ ]:
def modified_test_model_with_friends(index, image_friends_first_list, image_friends_second_list, label_first_list, label_second_list, face_embedder):
    # Get the embedding for the anchor image
    anchor_image = friends_path_first + image_friends_first_list[index]
    anchor_embedding = face_embedder.get_embedded_face(anchor_image)

    # Variables to keep track of most similar images
    distances = []  # A list to keep track of the distances

    # Iterate over images to compute distances
    for image in image_friends_second_list:
        # Get the embedding for this image
        image_embedding = face_embedder.get_embedded_face(friends_path_second + image)

        # Compute the distance
        if image_embedding is not None:
            distance = face_embedder._get_distance(anchor_embedding, image_embedding)
            distances.append(distance)
        else:
            distances.append(float('inf'))  # Handle the case where the face is not detected

    # Get the most similar images
    most_similar_images = face_embedder._get_most_similar_vactor(distances)

    # Check if the true label is in the top 3 predictions
    true_label = label_first_list[index]
    top_3_predictions = [label_second_list[img_index] for img_index, _ in most_similar_images[:3]]
    correct_prediction = true_label in top_3_predictions

    # Show the anchor image
    plt.figure(figsize=(20, 4))
    plt.subplot(1, 6, 1)
    plt.imshow(cv2.cvtColor(cv2.imread(anchor_image), cv2.COLOR_BGR2RGB))
    plt.title("Anchor image")
    plt.axis('off')

    # Show the most similar images
    for i, (img_index, similarity) in enumerate(most_similar_images):
        similar_image = cv2.imread(friends_path_second + image_friends_second_list[img_index])
        plt.subplot(1, 6, i + 2)
        plt.imshow(cv2.cvtColor(similar_image, cv2.COLOR_BGR2RGB))
        plt.title(f'Similarity: {similarity:.5f}%')
        plt.axis('off')
    plt.show()

    return correct_prediction

def calculate_accuracy(image_friends_first_list, image_friends_second_list, label_first_list, label_second_list, face_embedder):
    total_images = len(image_friends_first_list)
    correct_predictions = 0

    for index in range(total_images):
        correct_prediction = modified_test_model_with_friends(index, image_friends_first_list, image_friends_second_list, label_first_list, label_second_list, face_embedder)
        if correct_prediction:
            correct_predictions += 1

    accuracy = correct_predictions / total_images
    return accuracy

# Example usage:
# accuracy = calculate_accuracy(image_friends_first_list, image_friends_second_list, label_first_list, label_second_list, face_embedder)
# print(f'Accuracy: {accuracy * 100:.2f}%')

In [ ]:
accuracy = calculate_accuracy(friends_path_first_name, friends_path_second_name, label_first_list, label_second_list, embeding)
print(f'Accuracy: {accuracy * 100:.2f}%')

# 모델 경량화

In [ ]:
model = tf.keras.models.load_model('/content/drive/MyDrive/A.아이/github/models_folder/facenet/facenet_model.h5')

converter = tf.lite.TFLiteConverter.from_keras_model(model)

# FP16 양자화 설정
converter.target_spec.supported_types = [tf.float16]
converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_model = converter.convert()

# 변환된 모델을 .tflite 파일에 저장
open("/content/drive/MyDrive/A.아이/github/models_folder/facenet/FaceNet_Lite_model.tflite", "wb").write(tflite_model)